<a href="https://colab.research.google.com/github/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/extracao_fiat_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando todas bibliotecas necessárias

import requests
import zipfile
import io
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [4]:
# Carregando arquivo direto do site

# URL do arquivo .zip
url = 'https://dados.transportes.gov.br/dataset/12686da0-3d71-4499-b432-d270f785c907/resource/ee0d392c-c467-4002-884d-e89bd1f989f1/download/i_frota_por_uf_municipio_marca_e_modelo_ano_abril_2025.zip'

# Faz o download do arquivo com barra de progresso
print("Iniciando download do arquivo...")
response = requests.get(url, stream=True)  # stream=True para download em partes

# Verifica se o download foi bem-sucedido
if response.status_code == 200:
    # Obtém o tamanho total do arquivo (em bytes)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    # Cria um buffer para armazenar o conteúdo
    file_buffer = io.BytesIO()

    # Barra de progresso para o download
    with tqdm(total=total_size, unit='iB', unit_scale=True, desc="Download") as pbar:
        for data in response.iter_content(block_size):
            file_buffer.write(data)
            pbar.update(len(data))  # Atualiza a barra de progresso

    print("\nDownload concluído com sucesso!")

    # Prepara para extrair o arquivo
    file_buffer.seek(0)  # Volta ao início do buffer

    # Extrai o conteúdo do arquivo .zip com barra de progresso
    print("\nExtraindo arquivos...")
    with zipfile.ZipFile(file_buffer) as zip_file:
        # Obtém lista de arquivos para extração
        file_list = zip_file.infolist()

        # Barra de progresso para extração
        for file in tqdm(file_list, desc="Extraindo"):
            zip_file.extract(file, 'dados_extraidos')

    print("Extração concluída com sucesso!")

    # Renomeia o arquivo
    caminho_original = os.path.join('dados_extraidos', 'I_Frota_por_UF_Municipio_Marca_e_Modelo_Ano_Abril_2025.TXT')
    novo_nome = os.path.join('dados_extraidos', 'base_de_dados.csv')

    if os.path.exists(caminho_original):
        os.rename(caminho_original, novo_nome)
        print(f"Arquivo renomeado para: {novo_nome}")
    else:
        print("Arquivo original não encontrado.")
else:
    print(f"Erro ao baixar o arquivo: {response.status_code}")

Iniciando download do arquivo...


Download: 100%|██████████| 128M/128M [03:06<00:00, 689kiB/s]



Download concluído com sucesso!

Extraindo arquivos...


Extraindo: 100%|██████████| 1/1 [00:16<00:00, 16.05s/it]

Extração concluída com sucesso!
Arquivo renomeado para: dados_extraidos/base_de_dados.csv


In [3]:
# Abre o arquivo no modo de leitura
with open("dados_extraidos/base_de_dados.csv","r", encoding="utf-8") as arquivo:
    # Lê as 10 primeiras linhas
    for i in range(10):
        linha = arquivo.readline()
        # Exibe a linha atual
        print(linha.strip())

UF;Município;Marca Modelo;Ano Fabricação Veículo CRV;Qtd. Veículos
ACRE;ACRELANDIA;AGRALE/13000;2009; 1.0
ACRE;ACRELANDIA;AGRALE/1800;1989; 1.0
ACRE;ACRELANDIA;AGRALE/1800;1990; 1.0
ACRE;ACRELANDIA;AGRALE/1800D RD;1989; 1.0
ACRE;ACRELANDIA;AGRALE/1800D RD;1990; 2.0
ACRE;ACRELANDIA;AGRALE/1800D RD;1993; 1.0
ACRE;ACRELANDIA;AGRALE/6000 D;2005; 1.0
ACRE;ACRELANDIA;AGRALE/6000 D;2008; 1.0
ACRE;ACRELANDIA;AGRALE/6000 D;2010; 1.0


In [5]:
# Analisando o arquivo

# Caminho do arquivo
caminho_arquivo = os.path.join('dados_extraidos', 'base_de_dados.csv')

# Verifica se o arquivo existe
if os.path.exists(caminho_arquivo):
    print("Carregando o arquivo...")

    try:
        # Configuração do pandas para mostrar todas as colunas no info()
        pd.set_option('display.max_columns', None)

        # Carrega o arquivo em chunks (pedaços) para arquivos grandes
        chunks = pd.read_csv(caminho_arquivo,
                           encoding='utf-8',
                           delimiter=';',
                           chunksize=10000,
                           low_memory=False)

        # Concatena todos os chunks em um único DataFrame
        df = pd.concat(chunks)

        # Mostra apenas as informações do DataFrame
        print("\nInformações completas do DataFrame:")
        df.info(verbose=True, memory_usage='deep', show_counts=True)

    except Exception as e:
        print(f"\nErro ao ler o arquivo: {e}")
else:
    print("Arquivo não encontrado. Verifique o caminho.")

Carregando o arquivo...

Informações completas do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21446674 entries, 0 to 21446673
Data columns (total 5 columns):
 #   Column                      Non-Null Count     Dtype  
---  ------                      --------------     -----  
 0   UF                          21446674 non-null  object 
 1   Município                   21446674 non-null  object 
 2   Marca Modelo                21446674 non-null  object 
 3   Ano Fabricação Veículo CRV  21446674 non-null  object 
 4   Qtd. Veículos               21446674 non-null  float64
dtypes: float64(1), object(4)
memory usage: 5.6 GB


In [11]:
# Abrir o arquivo e ler as linhas
with open('dados_extraidos/base_de_dados.csv', 'r', encoding='utf-8') as arquivo:
    linhas = arquivo.readlines()

# Filtrar linhas com "FIAT" e "2025"
linhas_filtradas = [linha for linha in linhas if "FIAT" in linha and "2025" in linha]

# Adicionar cabeçalho manualmente (se necessário)
cabecalho = "Estado;Cidade;Modelo;Ano;Quantidade\n"
linhas_filtradas_com_cabecalho = [cabecalho] + linhas_filtradas

# Salvar em novo arquivo
with open("fiat_abril_2025.csv", 'w', encoding='utf-8') as novo_arquivo:
    novo_arquivo.writelines(linhas_filtradas_com_cabecalho)

print(f"Arquivo salvo em: fiat_abril_2025.csv")

Arquivo salvo em: fiat_abril_2025.csv


In [12]:
display(pd.read_csv("fiat_abril_2025.csv"))

,Estado;Cidade;Modelo;Ano;Quantidade
0,ACRE;ACRELANDIA;FIAT/ARGO DRIVE 1.0;2025; 1.0
1,ACRE;ACRELANDIA;FIAT/STRADA ULTRA T200AT;2025;...
2,ACRE;ACRELANDIA;FIAT/STRADA VOLCANO 13AT;2025;...
3,ACRE;ACRELANDIA;FIAT/STRADA VOLCANO CD13;2025;...
4,ACRE;ASSIS BRASIL;FIAT/STRADA VOLCANO CD13;202...
...,...
21869,TOCANTINS;TUPIRAMA;FIAT/TORO VOLC T270 AT;2025...
21870,TOCANTINS;WANDERLANDIA;FIAT/PULSE DRIVE AT;202...
21871,TOCANTINS;XAMBIOA;FIAT/STRADA RANCH T200AT;202...
21872,TOCANTINS;XAMBIOA;FIAT/STRADA VOLCANO 13AT;202...


In [13]:
# Analisando o arquivo

# Caminho do arquivo
caminho_arquivo = os.path.join('fiat_abril_2025.csv')

# Verifica se o arquivo existe
if os.path.exists(caminho_arquivo):
    print("Carregando o arquivo...")

    try:
        # Configuração do pandas para mostrar todas as colunas no info()
        pd.set_option('display.max_columns', None)

        # Carrega o arquivo em chunks (pedaços) para arquivos grandes
        chunks = pd.read_csv(caminho_arquivo,
                           encoding='utf-8',
                           delimiter=';',
                           chunksize=10000,
                           low_memory=False)

        # Concatena todos os chunks em um único DataFrame
        df = pd.concat(chunks)

        # Mostra apenas as informações do DataFrame
        print("\nInformações completas do DataFrame:")
        df.info(verbose=True, memory_usage='deep', show_counts=True)

    except Exception as e:
        print(f"\nErro ao ler o arquivo: {e}")
else:
    print("Arquivo não encontrado. Verifique o caminho.")

Carregando o arquivo...

Informações completas do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21874 entries, 0 to 21873
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Estado      21874 non-null  object 
 1   Cidade      21874 non-null  object 
 2   Modelo      21874 non-null  object 
 3   Ano         21874 non-null  int64  
 4   Quantidade  21874 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.8 MB
